In [34]:
def type_of_rating(data):
    facilities = []
    for row in data[1:]:
        for item in row:
            if isinstance(item, str):
                facility = item.lower()
                print(facility)
                if 'short term' in facility or 'long term / short term' in facility:
                    facilities.append('ST')
                    break
                elif 'long term' in facility or 'long-term' in facility or 'long - term' in facility:
                    facilities.append('LT')
                    break
                elif 'unallocated term' in facility:
                    facilities.append('UT')
                    break
        else:
            facilities.append('ST')
    return facilities


# def outlook(data):
#     ratings = []
#     for row in data[1:]:
#         rating = row[2]
#         if 'Stable' in rating:
#             ratings.append('Stable')
#         elif 'Positive' in rating or 'Negative' in rating:
#             ratings.append(rating.split(';')[1].strip())
#         else:
#             ratings.append('-')

#     return ratings


from PyPDF2 import PdfReader
import numpy as np

def remove_commas(data):
    for row in data:
        for i in range(len(row)):
            if isinstance(row[i], str) and row[i].replace(',', '').replace('.', '').isdigit():
                row[i] = row[i].replace(',', '')
    return data

from PyPDF2 import PdfReader
def extract_text_from_pdf(pdf_file_path):
    reader = PdfReader(pdf_file_path)
    page = reader.pages[0]
    text = page.extract_text()
    return text

def find_agency_names(pdf_file_path):
    text = extract_text_from_pdf(pdf_file_path)
    agencies = []
    if 'CARE' in text:
        agencies.append('CARE')
    if 'ICRA' in text:
        agencies.append('ICRA')
    if 'Brickwork rating' in text or 'brickworkrating' in text:
        agencies.append('Brickworks')
    return agencies

In [35]:

# # ## code for contact details

import re
from PyPDF2 import PdfReader

def extract_text_from_last_pages(pdf_file_path, num_pages=3):
    reader = PdfReader(pdf_file_path)
    text = ""
    total_pages = len(reader.pages)
    for page_num in range(total_pages - num_pages, total_pages):
        text += reader.pages[page_num].extract_text()
    return text


import re
from PyPDF2 import PdfReader

def name_desi(name):
    for i in name:
        t = (i.split('\n'))
        name = t[0]
        desi = t[1]
    return name,desi


def extract_text_from_last_pages(pdf_file_path, num_pages=3):
    reader = PdfReader(pdf_file_path)
    text = ""
    total_pages = len(reader.pages)
    for page_num in range(total_pages - num_pages, total_pages):
        text += reader.pages[page_num].extract_text()
    return text

def extract_analyst_contacts(pdf_file_path):
    text = extract_text_from_last_pages(pdf_file_path)
    if "Analytical Contacts" in text:
        name_pattern = re.compile(r'(?<=Analytical Contacts\s)[A-Za-z\s]+')
        phone_pattern = re.compile(r'\+\d{2}\s\d{2}\s\d{4}\s\d{4}')
        email_pattern = re.compile(r'[\w\.-]+@[\w\.-]+')
        names = name_pattern.findall(text)
        phones = phone_pattern.findall(text)
        emails = email_pattern.findall(text)
        
        names_list = []
        emails_list = []
        contact_numbers_list = []
        desig = []
        for name in names:
            name = name.strip()
            name1 = [name.strip()]
            nam,desi = name_desi(name1)
            phone_match = phone_pattern.search(text, text.find(name))
            phone = phone_match.group(0) if phone_match else None
            email_match = email_pattern.search(text, text.find(name))
            email = email_match.group(0) if email_match else None
            
            names_list.append(nam)
            desig.append(desi)
            contact_numbers_list.append(phone)
            emails_list.append(email)

        return names_list, desig, emails_list, contact_numbers_list
    else:
        return ['-'], ['-'], ['-']



In [36]:
# def extract_all_amount(data):
#     short_term_amounts = []
#     long_term_amounts = []
#     unallocated_amounts = []
    
#     for row in data:
#         if isinstance(row[1], str) and row[1].replace('.', '', 1).isdigit():
#             amount = float(row[1])
#             for item in row:
#                 if isinstance(item, str):
#                     facility = item.lower()
                    
#                     if 'short term' in facility or 'long term / short term' in facility or 'short-term' in facility:
#                         short_term_amounts.append(amount)
#                         long_term_amounts.append(0.00)
#                         unallocated_amounts.append(0.00)
#                         break
#                     elif 'long term' in facility or 'long-term' in facility:
#                         long_term_amounts.append(amount)
#                         short_term_amounts.append(0.00)
#                         unallocated_amounts.append(0.00)
#                         break
#                     elif 'lc/bg' in facility or 'LC/BG' in facility:
#                         short_term_amounts.append(amount)
#                         long_term_amounts.append(0.00)
#                         unallocated_amounts.append(0.00)
#                         break
#                     elif 'term loan' in facility or 'Term loan' in facility:
#                         long_term_amounts.append(amount)
#                         short_term_amounts.append(0.00)
#                         unallocated_amounts.append(0.00)
#                         break
#                     elif 'unallocated term' in facility or 'propose' in facility:
#                         unallocated_amounts.append(amount)
#                         short_term_amounts.append(0.00)
#                         long_term_amounts.append(0.00)
#                         break
#             else:
#                 short_term_amounts.append(amount)
#                 unallocated_amounts.append(0.00)
#                 long_term_amounts.append(0.00)
    
#     return short_term_amounts, long_term_amounts, unallocated_amounts


# current amount
def extract_all_amount(data):
    short_term_amounts = []
    long_term_amounts = []
    unallocated_amounts = []
    
    for row in data:
        if isinstance(row[2], str) and row[2].replace('.', '', 1).isdigit():
            amount = float(row[2]) 
        else:
            amount = 0.00
        
        facility1 = row[0]
        facility = facility1.lower()
        
        if 'short term' in facility or 'long term / short term' in facility or 'short term  nonfund based limits  lc/bg' in facility:
            short_term_amounts.append(amount)
            long_term_amounts.append(0.00)
            unallocated_amounts.append(0.00)
        elif 'long term' in facility or 'long-term' in facility or 'long - term' in facility or 'term loan' in facility or 'fund based  term loan' in facility or 'long term fundbased limits  proposed term loans' in facility or 'long term  fundbased limits  cash credit' in facility:
            long_term_amounts.append(amount)
            short_term_amounts.append(0.00)
            unallocated_amounts.append(0.00)
        elif 'lc/bg' in facility or 'LC/BG' in facility:
            short_term_amounts.append(amount)
            long_term_amounts.append(0.00)
            unallocated_amounts.append(0.00)
        elif 'term loan' in facility or 'Term loan' in facility:
            long_term_amounts.append(amount)
            short_term_amounts.append(0.00)
            unallocated_amounts.append(0.00)
        elif 'unallocated term' in facility or 'propose' in facility:
            unallocated_amounts.append(amount)
            short_term_amounts.append(0.00)
            long_term_amounts.append(0.00)
        else:
            short_term_amounts.append(amount)
            unallocated_amounts.append(0.00)
            long_term_amounts.append(0.00)
    
    return short_term_amounts, long_term_amounts, unallocated_amounts


In [37]:
def outlook_br(data):
    ratings = []
    for row in data[1:]:
        for item in row:
            if isinstance(item, str) and 'Stable' in item:
                ratings.append('Stable')
                break
            elif isinstance(item, str) and ('Positive' in item or 'Negative' in item):
                ratings.append(item.split(';')[1].strip())
                break
        else:
            ratings.append('-')
    
    return ratings

# outlook_br(filtered_previous)

# def extract_ratings_from_data_br(data):
#     rating_pattern = re.compile(r'\b(?:[A-Z]{1,3}[+-]?/[A-Z0-9+-]+(?:/[A-Z0-9+-]+)?(?:;Negative)?)\b|[A-Z0-9+-]+(?:;Negative)?')
#     ratings = []
#     for row in data:
#         row_ratings = []
#         for cell in row:
#             if isinstance(cell, str): 
#                 matches = rating_pattern.findall(cell)
#                 valid_matches = [match for match in matches if len(match) > 1 and not match.isdigit() and match not in ['CARE', 'ICRA', 'Commercial paper (CP)','CP','CE','NCD','BWR','II']]
#                 row_ratings.extend(valid_matches)
#         if row_ratings:
#             ratings.append('/'.join(row_ratings))
    
#     return ratings


def extract_ratings_from_data(data):
    rating_pattern = re.compile(r'\b(?:[A-Z]{1,3}[+-]?(?:\([Nn]egative\))?)\b|[A-Z0-9+-]+(?:\([Nn]egative\))?')
    ratings = []
    for row in data:
        row_ratings = []
        for cell in row:
            if isinstance(cell, str): 
                matches = rating_pattern.findall(cell)
                valid_matches = [match for match in matches if len(match) > 1 and not match.isdigit() and match not in ['CARE', 'ICRA', 'Commercial paper (CP)', 'CP', 'CE', 'BD', 'LC', 'BG', 'LC/BG']]
                row_ratings.extend(valid_matches)
        if row_ratings:
            ratings.append('/'.join(row_ratings))
        else:
            ratings.append('-')
    
    return ratings




# extract_ratings_from_data_br(filtered_previous)

def extract_type_of_facilities(data):
    facilities = []
    pattern = re.compile(r'Commercial paper \(CP\)|^[A-Za-z\s\-\/\(\)]+(?:Bank Facilities|Term Loan|Commercial paper|CE rating is based on the pledge of shares of|Amount)$')
    long_term_pattern = re.compile(r'\bLong Term\b')  
    for row in data:
        if isinstance(row[0], str) and pattern.match(row[0]):
            facilities.append(row[0])
        for item in row:
            if isinstance(item, str) and long_term_pattern.search(item):
                facilities.append(item)
    return facilities
# extract_type_of_facilities(filtered_previous)

def extract_company_names(text):
    company_name_pattern = re.compile(r'\b[A-Z][a-zA-Z\s]*(?:\([^)]*\))?\s*(?:Ltd|Inc|Corp|LLC|PLC|Pvt\. Ltd)\b')
    matches = company_name_pattern.findall(text)
    return matches if matches else ["Company name not found"]

def adjust_rat_length(rat, rating_type):
    rat1 = []
    if len(rat) != len(rating_type):
        rat1 = rat + [rat[-1]] * (len(rating_type) - len(rat))
    else:
        rat1 = rat
    return rat1

In [38]:
def outlook_br(data):
    ratings = []
    for row in data[0:]:
        for item in row:
            if isinstance(item, str) and 'Stable' in item:
                ratings.append('Stable')
                break
            elif isinstance(item, str) and ('Positive' in item or 'Negative' in item):
                ratings.append(item.split(';')[1].strip())
                break
        else:
            ratings.append('-')
    
    return ratings


def type_of_rating(data):
    facilities = []
    for row in data[0:]:
        for item in row:
            if isinstance(item, str):
                facility = item.lower()
                print(facility)
                if 'short term' in facility or 'long term / short term' in facility:
                    facilities.append('ST')
                    break
                elif 'long term' in facility or 'long-term' in facility or 'long - term' in facility:
                    facilities.append('LT')
                    break
                elif 'unallocated term' in facility:
                    facilities.append('UT')
                    break
        else:
            facilities.append('ST')
    return facilities

In [39]:
import pandas as pd
# xl_path = r"D:\Python\ajit\project_credit_rating\try\excel\Adani Infra (India) Ltd.xlsx"
# pdf_file_path = r"D:\Python\ajit\project_credit_rating\try\pdf\Adani Infra (India) Ltd.pdf"

pdf_file_path = r"D:\Python\ajit\project_credit_rating\try\auto\brickwork\pdf\Adani Infrastructure Management Services Ltd.pdf"
xl_path = r"D:\Python\ajit\project_credit_rating\try\auto\brickwork\excel\Adani Infrastructure Management Services Ltd.xlsx"


com1 = []
age=[]
ana_name = []
desi = []
email = []
contact_number = []
ty_faci = []
xls = pd.ExcelFile(xl_path)
table_name = xls.sheet_names
df1 = pd.read_excel(xls, sheet_name="Page 1 - Table 1", header=None)
a1 = df1.values.tolist()

filtered_rs = [row for row in a1 if '(Rs. crore)' not in row]
filtered_total = [row for row in filtered_rs if 'Total' not in row]
filtered_pre = [row for row in filtered_total if 'Previous' not in row]  
filtered_prev = [row for row in filtered_pre if 'Present Rating' not in row]
filtered_previous = [row for row in filtered_prev if 'Present Rating#' not in row]



rating_type = type_of_rating(filtered_previous)
outlook1 = outlook_br(filtered_previous)
short_term_amounts, long_term_amounts, unallocated_amounts = extract_all_amount(filtered_previous)
ratings = extract_ratings_from_data_br(filtered_previous)
rat1 = adjust_rat_length(ratings, rating_type)
type_facility=extract_type_of_facilities(filtered_previous)
ty_faci = adjust_rat_length(type_facility, rating_type)
name, desig,emails, contact_numbers = extract_analyst_contacts(pdf_file_path)
# name = [name1.replace('Senior Manager', '').strip() for name1 in names]
ana_name.extend(name*len(rating_type))
desi.extend(desig*len(rating_type))

email.extend(emails*len(rating_type))
contact_number.extend(contact_numbers*len(rating_type))
agencies = find_agency_names(pdf_file_path)
text = extract_text_from_pdf(pdf_file_path)
text1 = (text[0:100])
company_names = extract_company_names(text1)

com1.extend(company_names * len(rating_type))
age.extend(agencies * len(rating_type))

print(len(com1),len(age),len(ty_faci),len(rating_type),len(outlook1),len(rat1),len(short_term_amounts),
      len(long_term_amounts),len(unallocated_amounts),len(ana_name),len(desi),len(email),len(contact_number))

data = {
        'Name of Adani Entity': com1,
        'Name of the Rating Agency': age,
        'Types of Rating': rating_type,
        'Ratings': rat1,
        'Outlook': outlook1,
        'Type of Facility': ty_faci,
        'Rated Outstanding Long Term Amount': long_term_amounts,
        'Rated Outstanding Short Term Amount': short_term_amounts,
        'Rated Outstanding Unallocated Amount': unallocated_amounts,
        'Credit Rating Analyst Name': ana_name,
        'Credit Rating Analyst Designation':desi,
        'Credit Rating Analyst Email-Id': email,
        'Credit Rating Analyst Contact': contact_number
        }

# Create the DataFrame
df = pd.DataFrame(data)
df

(1) term loan i
473.45
473.45
long term
(2) term loan ii
114.00
-
long term
2 2 2 2 2 2 2 2 2 2 2 2 2


,Name of Adani Entity,Name of the Rating Agency,Types of Rating,Ratings,Outlook,Type of Facility,Rated Outstanding Long Term Amount,Rated Outstanding Short Term Amount,Rated Outstanding Unallocated Amount,Credit Rating Analyst Name,Credit Rating Analyst Designation,Credit Rating Analyst Email-Id,Credit Rating Analyst Contact
0,Adani Infrastructur e Management Services Ltd,Brickworks,LT,AA-/BBB-,Stable,Long Term,473.45,0.0,0.0,Forum R Parekh,Senior Manager,forum.rp@brickworkratings.com,+91 22 6745 6621
1,Adani Infrastructur e Management Services Ltd,Brickworks,LT,AA-,Stable,Long Term,0.00,0.0,0.0,Forum R Parekh,Senior Manager,forum.rp@brickworkratings.com,+91 22 6745 6621


In [25]:
((com1),(age),(ty_faci),(rating_type),(outlook1),(rat1),(short_term_amounts),
      (long_term_amounts),(unallocated_amounts),(ana_name),(email),(contact_number))

(['Adani Infrastructur e Management Services Ltd',
  'Adani Infrastructur e Management Services Ltd'],
 ['Brickworks', 'Brickworks'],
 ['Long Term', 'Long Term'],
 ['LT', 'LT'],
 ['Stable', 'Stable'],
 ['AA-/BBB-', 'AA-'],
 [0.0, 0.0],
 [473.45, 0.0],
 [0.0, 0.0],
 ['Forum R Parekh ', 'Forum R Parekh '],
 ['forum.rp@brickworkratings.com', 'forum.rp@brickworkratings.com'],
 ['+91 22 6745 6621', '+91 22 6745 6621'])

In [27]:

# # current amount
# def extract_all_amount(data):
#     short_term_amounts = []
#     long_term_amounts = []
#     unallocated_amounts = []
    
#     for row in data:
#         if isinstance(row[2], str) and row[2].replace('.', '', 1).isdigit():
#             amount = float(row[2]) 
#         else:
#             amount = 0.00
        
#         facility1 = row[0]
#         facility = facility1.lower()
        
#         if 'short term' in facility or 'long term / short term' in facility or 'short term  nonfund based limits  lc/bg' in facility:
#             short_term_amounts.append(amount)
#             long_term_amounts.append(0.00)
#             unallocated_amounts.append(0.00)
#         elif 'long term' in facility or 'long-term' in facility or 'long - term' in facility or 'term loan' in facility or 'fund based  term loan' in facility or 'long term fundbased limits  proposed term loans' in facility or 'long term  fundbased limits  cash credit' in facility:
#             long_term_amounts.append(amount)
#             short_term_amounts.append(0.00)
#             unallocated_amounts.append(0.00)
#         elif 'lc/bg' in facility or 'LC/BG' in facility:
#             short_term_amounts.append(amount)
#             long_term_amounts.append(0.00)
#             unallocated_amounts.append(0.00)
#         elif 'term loan' in facility or 'Term loan' in facility:
#             long_term_amounts.append(amount)
#             short_term_amounts.append(0.00)
#             unallocated_amounts.append(0.00)
#         elif 'unallocated term' in facility or 'propose' in facility:
#             unallocated_amounts.append(amount)
#             short_term_amounts.append(0.00)
#             long_term_amounts.append(0.00)
#         else:
#             short_term_amounts.append(amount)
#             unallocated_amounts.append(0.00)
#             long_term_amounts.append(0.00)
    
#     return short_term_amounts, long_term_amounts, unallocated_amounts


In [23]:
filtered_previous

[['NCD*\n(unlisted)(1)',
  '250',
  '-',
  'Long Term',
  'BWR AA- (CE)\n(Stable)',
  'Withdrawn on Redemption']]

In [26]:
filtered_previous

[['(1) Term Loan I',
  '473.45',
  '473.45',
  'Long Term',
  'BWR AA- (CE)\n(1) (Stable)',
  'BWR BBB- (Stable) and\n(1) Withdrawn\n(Downgrade and\nWithdrawn)'],
 ['(2) Term Loan II',
  '114.00',
  '-',
  'Long Term',
  'BWR AA- (CE)\n(2) (Stable)',
  'Withdrawn on full\nrepayment']]

In [328]:
short_term_amounts, long_term_amounts, unallocated_amounts = extract_all_amount(filtered_previous)

In [329]:
short_term_amounts,long_term_amounts,unallocated_amounts

([0.0, 0.0, 0.0], [0.0, 473.45, 0.0], [0.0, 0.0, 0.0])

In [294]:
short_term_amounts, long_term_amounts, unallocated_amounts = extract_all_amount(filtered_previous)

In [295]:
short_term_amounts,long_term_amounts,unallocated_amounts

([], [], [])

In [293]:
def extract_all_amount(data):
    short_term_amounts = []
    long_term_amounts = []
    unallocated_amounts = []
    
    for row in data:
        if isinstance(row[2], str) and row[2].replace('.', '', 1).isdigit():
            amount = float(row[2])
            for item in row:
                if isinstance(item, str):
                    facility = item.lower()
                    
                    if 'short term' in facility or 'long term / short term' in facility or 'short-term' in facility:
                        short_term_amounts.append(amount)
                        long_term_amounts.append(0.00)
                        unallocated_amounts.append(0.00)
                        break
                    elif 'long term' in facility or 'long-term' in facility:
                        long_term_amounts.append(amount)
                        short_term_amounts.append(0.00)
                        unallocated_amounts.append(0.00)
                        break
                    elif 'lc/bg' in facility or 'LC/BG' in facility:
                        short_term_amounts.append(amount)
                        long_term_amounts.append(0.00)
                        unallocated_amounts.append(0.00)
                        break
                    elif 'term loan' in facility or 'Term loan' in facility:
                        long_term_amounts.append(amount)
                        short_term_amounts.append(0.00)
                        unallocated_amounts.append(0.00)
                        break
                    elif 'unallocated term' in facility or 'propose' in facility:
                        unallocated_amounts.append(amount)
                        short_term_amounts.append(0.00)
                        long_term_amounts.append(0.00)
                        break
            else:
                short_term_amounts.append(amount)
                unallocated_amounts.append(0.00)
                long_term_amounts.append(0.00)
    
    return short_term_amounts, long_term_amounts, unallocated_amounts

In [305]:
filtered_previous

[['Facility /\nInstrument*',
  'Amt Rated\n(Rs. Cr)',
  nan,
  'Tenor',
  'Previous Ratings#\n(October 21, 2022)',
  'Present Rating#'],
 ['NCD*\n(unlisted)(1)',
  '250',
  '-',
  'Long Term',
  'BWR AA- (CE)\n(Stable)',
  'Withdrawn on Redemption']]

In [299]:
for row in filtered_previous:
    if isinstance(row[1], str) and row[1].replace('.', '', 1).isdigit():
        amount = float(row[1])
        print(amount)
        # for item in row:
        #     if isinstance(item, str):
        #         facility = item.lower()
                
        #         if 'short term' in facility or 'long term / short term' in facility or 'short-term' in facility:
        #             short_term_amounts.append(amount)
        #             long_term_amounts.append(0.00)
        #             unallocated_amounts.append(0.00)
        #             break
        #         elif 'long term' in facility or 'long-term' in facility:
        #             long_term_amounts.append(amount)
        #             short_term_amounts.append(0.00)
        #             unallocated_amounts.append(0.00)
        #             break
        #         elif 'lc/bg' in facility or 'LC/BG' in facility:
        #             short_term_amounts.append(amount)
        #             long_term_amounts.append(0.00)
        #             unallocated_amounts.append(0.00)
        #             break
        #         elif 'term loan' in facility or 'Term loan' in facility:
        #             long_term_amounts.append(amount)
        #             short_term_amounts.append(0.00)
        #             unallocated_amounts.append(0.00)
        #             break
        #         elif 'unallocated term' in facility or 'propose' in facility:
        #             unallocated_amounts.append(amount)
        #             short_term_amounts.append(0.00)
        #             long_term_amounts.append(0.00)
        #             break
        # else:
        #     short_term_amounts.append(amount)
        #     unallocated_amounts.append(0.00)
        #     long_term_amounts.append(0.00)

250.0


In [283]:

# # # ## code for contact details

# import re
# from PyPDF2 import PdfReader

# def extract_text_from_last_pages(pdf_file_path, num_pages=3):
#     reader = PdfReader(pdf_file_path)
#     text = ""
#     total_pages = len(reader.pages)
#     for page_num in range(total_pages - num_pages, total_pages):
#         text += reader.pages[page_num].extract_text()
#     return text

# def extract_analyst_contacts(pdf_file_path):
#     text = extract_text_from_last_pages(pdf_file_path)
#     if "Analytical Contacts" in text:
#         name_pattern = re.compile(r'(?<=Analytical Contacts\s)[A-Za-z\s]+')
#         phone_pattern = re.compile(r'\+\d{2}\s\d{2}\s\d{4}\s\d{4}')
#         email_pattern = re.compile(r'[\w\.-]+@[\w\.-]+')
#         names = name_pattern.findall(text)
#         phones = phone_pattern.findall(text)
#         emails = email_pattern.findall(text)
        
#         names_list = []
#         emails_list = []
#         contact_numbers_list = []
#         desig = []
#         for name in names:
#             name = name.strip()
#             name1 = [name.strip()]
#             nam,desi = name_desi(name1)
#             phone_match = phone_pattern.search(text, text.find(name))
#             phone = phone_match.group(0) if phone_match else None
#             email_match = email_pattern.search(text, text.find(name))
#             email = email_match.group(0) if email_match else None
            
#             names_list.append(nam)
#             desig.append(desi)
#             contact_numbers_list.append(phone)
#             emails_list.append(email)

#         return names_list, desig, emails_list, contact_numbers_list
#     else:
#         return ['-'], ['-'], ['-']



In [145]:
# def type_of_rating(data):
#     facilities = []
#     for row in data[1:]:
#         for item in row:
#             if isinstance(item, str):
#                 facility = item.lower()
#                 print(facility)
#                 if 'short term' in facility or 'long term / short term' in facility:
#                     facilities.append('ST')
#                     break
#                 elif 'long term' in facility or 'long-term' in facility or 'long - term' in facility:
#                     facilities.append('LT')
#                     break
#                 elif 'unallocated term' in facility:
#                     facilities.append('UT')
#                     break
#         else:
#             facilities.append('ST')
#     return facilities

In [130]:
name

['Forum R Parekh']

In [109]:
filtered_previous

[['Facility /\nInstrument*',
  'Amt Rated\n(Rs. Cr)',
  nan,
  'Tenor',
  'Previous Ratings#\n(October 21, 2022)',
  'Present Rating#'],
 ['NCD*\n(unlisted)(1)',
  '250',
  '-',
  'Long Term',
  'BWR AA- (CE)\n(Stable)',
  'Withdrawn on Redemption']]

In [135]:
# import re

# def extract_all_amount(data):
#     short_term_amounts = []
#     long_term_amounts = []
#     unallocated_amounts = []
    
#     for row in data:
#         if isinstance(row[1], str) and row[1].replace('.', '', 1).isdigit():
#             amount = float(row[1])
#             for item in row:
#                 if isinstance(item, str):
#                     facility = item.lower()
                    
#                     if 'short term' in facility or 'long term / short term' in facility or 'short-term' in facility:
#                         short_term_amounts.append(amount)
#                         long_term_amounts.append(0.00)
#                         unallocated_amounts.append(0.00)
#                         break
#                     elif 'long term' in facility or 'long-term' in facility:
#                         long_term_amounts.append(amount)
#                         short_term_amounts.append(0.00)
#                         unallocated_amounts.append(0.00)
#                         break
#                     elif 'lc/bg' in facility or 'LC/BG' in facility:
#                         short_term_amounts.append(amount)
#                         long_term_amounts.append(0.00)
#                         unallocated_amounts.append(0.00)
#                         break
#                     elif 'term loan' in facility or 'Term loan' in facility:
#                         long_term_amounts.append(amount)
#                         short_term_amounts.append(0.00)
#                         unallocated_amounts.append(0.00)
#                         break
#                     elif 'unallocated term' in facility or 'propose' in facility:
#                         unallocated_amounts.append(amount)
#                         short_term_amounts.append(0.00)
#                         long_term_amounts.append(0.00)
#                         break
#             else:
#                 short_term_amounts.append(amount)
#                 unallocated_amounts.append(0.00)
#                 long_term_amounts.append(0.00)
    
#     return short_term_amounts, long_term_amounts, unallocated_amounts

# # Example usage
# data = [['Facility /\nInstrument*', 'Amt Rated\n(Rs. Cr)', None, 'Tenor', 'Previous Ratings#\n(October 21, 2022)', 'Present Rating#'],
#         ['NCD*\n(unlisted)(1)', '250', '-', 'Long Term', 'BWR AA- (CE)\n(Stable)', 'Withdrawn on Redemption']]

# short_term_amounts, long_term_amounts, unallocated_amounts = extract_all_amount(data)
# print(f"Short Term Amounts: {short_term_amounts}")
# print(f"Long Term Amounts: {long_term_amounts}")
# print(f"Unallocated Amounts: {unallocated_amounts}")


In [ ]:
import re
from PyPDF2 import PdfReader

def extract_text_from_last_pages(pdf_file_path, num_pages=3):
    reader = PdfReader(pdf_file_path)
    text = ""
    total_pages = len(reader.pages)
    for page_num in range(total_pages - num_pages, total_pages):
        text += reader.pages[page_num].extract_text()
    return text

def extract_analyst_contacts(pdf_file_path):
    text = extract_text_from_last_pages(pdf_file_path)
    
    if "Analytical Contacts" in text:
        name_pattern = re.compile(r'(?<=Analytical Contacts\s)[A-Za-z\s]+')
        phone_pattern = re.compile(r'\+\d{2}\s\d{2}\s\d{4}\s\d{4}')
        email_pattern = re.compile(r'[\w\.-]+@[\w\.-]+')
        
        names = name_pattern.findall(text)
        phones = phone_pattern.findall(text)
        emails = email_pattern.findall(text)
        
        names_list = []
        emails_list = []
        contact_numbers_list = []
        
        for name in names:
            name = name.replace('Senior Manager', '').strip()  # Remove 'Senior Manager'
            phone_match = phone_pattern.search(text, text.find(name))
            phone = phone_match.group(0) if phone_match else None
            email_match = email_pattern.search(text, text.find(name))
            email = email_match.group(0) if email_match else None
            
            names_list.append(name)
            contact_numbers_list.append(phone)
            emails_list.append(email)

        return names_list, emails_list, contact_numbers_list
    else:
        return ['-'], ['-'], ['-']

# Example usage (assuming pdf_file_path is defined)
# names, emails, contact_numbers = extract_analyst_contacts(pdf_file_path)

# For demonstration purposes, let's use the provided data
names_list = ['Forum R Parekh \nSenior Manager']
emails_list = ['forum.rp@brickworkratings.com']
contact_numbers_list = ['+91 22 6745 6621']

# Remove 'Senior Manager' from the names list
names_list = [name.replace('Senior Manager', '').strip() for name in names_list]

print(f"Names: {names_list}")
print(f"Emails: {emails_list}")
print(f"Contact Numbers: {contact_numbers_list}")


In [100]:
names, emails, contact_numbers = extract_analyst_contacts(pdf_file_path)

In [101]:
name,emails,contact_numbers

(['-'], ['-'], ['-'])

In [71]:
a1

[['Facility /\nInstrument*',
  'Amt Rated\n(Rs. Cr)',
  nan,
  'Tenor',
  'Previous Ratings#\n(October 21, 2022)',
  'Present Rating#'],
 ['nan', 'Previous', 'Present', nan, nan, nan],
 ['NCD*\n(unlisted)(1)',
  '250',
  '-',
  'Long Term',
  'BWR AA- (CE)\n(Stable)',
  'Withdrawn on Redemption']]

In [72]:
filtered_rs = [row for row in a1 if '(Rs. crore)' not in row]
filtered_total = [row for row in filtered_rs if 'Total' not in row]
filtered_previous = [row for row in filtered_total if 'Previous' not in row]


In [73]:
filtered_previous

[['Facility /\nInstrument*',
  'Amt Rated\n(Rs. Cr)',
  nan,
  'Tenor',
  'Previous Ratings#\n(October 21, 2022)',
  'Present Rating#'],
 ['NCD*\n(unlisted)(1)',
  '250',
  '-',
  'Long Term',
  'BWR AA- (CE)\n(Stable)',
  'Withdrawn on Redemption']]

In [144]:
# def outlook_br(data):
#     ratings = []
#     for row in data[1:]:
#         for item in row:
#             if isinstance(item, str) and 'Stable' in item:
#                 ratings.append('Stable')
#                 break
#             elif isinstance(item, str) and ('Positive' in item or 'Negative' in item):
#                 ratings.append(item.split(';')[1].strip())
#                 break
#         else:
#             ratings.append('-')
    
#     return ratings

# outlook_br(filtered_previous)

# def extract_ratings_from_data_br(data):
#     rating_pattern = re.compile(r'\b(?:[A-Z]{1,3}[+-]?/[A-Z0-9+-]+(?:/[A-Z0-9+-]+)?(?:;Negative)?)\b|[A-Z0-9+-]+(?:;Negative)?')
#     ratings = []
#     for row in data:
#         row_ratings = []
#         for cell in row:
#             if isinstance(cell, str): 
#                 matches = rating_pattern.findall(cell)
#                 valid_matches = [match for match in matches if len(match) > 1 and not match.isdigit() and match not in ['CARE', 'ICRA', 'Commercial paper (CP)','CP','CE','NCD','BWR']]
#                 row_ratings.extend(valid_matches)
#         if row_ratings:
#             ratings.append('/'.join(row_ratings))
    
#     return ratings

# extract_ratings_from_data_br(filtered_previous)

# def extract_type_of_facilities(data):
#     facilities = []
#     pattern = re.compile(r'Commercial paper \(CP\)|^[A-Za-z\s\-\/\(\)]+(?:Bank Facilities|Term Loan|Commercial paper|CE rating is based on the pledge of shares of|Amount)$')
#     long_term_pattern = re.compile(r'\bLong Term\b')  
#     for row in data:
#         if isinstance(row[0], str) and pattern.match(row[0]):
#             facilities.append(row[0])
#         for item in row:
#             if isinstance(item, str) and long_term_pattern.search(item):
#                 facilities.append(item)
#     return facilities
# extract_type_of_facilities(filtered_previous)

# def extract_company_names_br(text):
#     company_name_pattern = re.compile(r'\b[A-Z][a-zA-Z\s]*(?:\([^)]*\))?\s*(?:Ltd|Inc|Corp|LLC|PLC|Pvt\. Ltd)\b')
#     matches = company_name_pattern.findall(text)
#     return matches if matches else ["Company name not found"]

# text1 = (text[0:100])
# company_names = extract_company_names(text1)
# print(f"Extracted Company Names: {company_names}")

In [143]:

# def extract_ratings_from_data_br(data):
#     rating_pattern = re.compile(r'\b(?:[A-Z]{1,3}[+-]?/[A-Z0-9+-]+(?:/[A-Z0-9+-]+)?(?:;Negative)?)\b|[A-Z0-9+-]+(?:;Negative)?')
#     ratings = []
#     for row in data:
#         row_ratings = []
#         for cell in row:
#             if isinstance(cell, str): 
#                 matches = rating_pattern.findall(cell)
#                 valid_matches = [match for match in matches if len(match) > 1 and not match.isdigit() and match not in ['CARE', 'ICRA', 'Commercial paper (CP)','CP','CE','NCD','BWR']]
#                 row_ratings.extend(valid_matches)
#         if row_ratings:
#             ratings.append('/'.join(row_ratings))
    
#     return ratings

# extract_ratings_from_data_br(filtered_previous)

In [142]:
# def extract_type_of_facilities(data):
#     facilities = []
#     pattern = re.compile(r'Commercial paper \(CP\)|^[A-Za-z\s\-\/\(\)]+(?:Bank Facilities|Term Loan|Commercial paper|CE rating is based on the pledge of shares of|Amount)$')
#     long_term_pattern = re.compile(r'\bLong Term\b')  
#     for row in data:
#         if isinstance(row[0], str) and pattern.match(row[0]):
#             facilities.append(row[0])
#         for item in row:
#             if isinstance(item, str) and long_term_pattern.search(item):
#                 facilities.append(item)
#     return facilities
# extract_type_of_facilities(filtered_previous)

In [140]:
def extract_type_of_facilities_br(data):
    facilities = []
    pattern = re.compile(r'Commercial paper \(CP\)|^[A-Za-z\s\-\/\(\)]+(?:Bank Facilities|Term Loan|Commercial paper|CE rating is based on the pledge of shares of|Amount)$')
    long_term_pattern = re.compile(r'\bLong Term\b')  
    for row in data:
        if isinstance(row[0], str) and pattern.match(row[0]):
            facilities.append(row[0])
        for item in row:
            if isinstance(item, str) and long_term_pattern.search(item):
                facilities.append(item)
    return facilities



In [139]:
# pdf_file_path = r"D:\Python\ajit\project_credit_rating\try\pdf\Adani Infra (India) Ltd.pdf"
# text1 = (text[0:100])
# company_names = extract_company_names(text1)
# print(f"Extracted Company Names: {company_names}")

In [138]:
# def extract_type_of_facilities(data):
#     facilities = []
#     pattern = re.compile(r'Commercial paper \(CP\)|^[A-Za-z\s\-\/\(\)]+(?:Bank Facilities|Term Loan|Commercial paper|CE rating is based on the pledge of shares of|Amount)$')
#     long_term_pattern = re.compile(r'\bLong Term\b')  
#     for row in data:
#         if isinstance(row[0], str) and pattern.match(row[0]):
#             facilities.append(row[0])
#         for item in row:
#             if isinstance(item, str) and long_term_pattern.search(item):
#                 facilities.append(item)
#     return facilities
# extract_type_of_facilities(data)